In [1]:
import cv2
import numpy as np
import os

In [2]:
path = r"D:\MAMAA2\Aging\Code\ImageCpy\check"
file_list = [path + "\\" + i for i in os.listdir(path)]

filter_temp = cv2.cvtColor(cv2.imread('temp1.jpg'),cv2.COLOR_BGR2GRAY)
ret1, filter_temp = cv2.threshold(filter_temp, 127, 255, cv2.THRESH_BINARY)
kernel = np.ones((7,7),np.uint8)
scale = 0.5

pts = [[507,1070],[646,1110],[595,1287],[478,1254]]
for img_path in file_list:
    img = cv2.imread(img_path)
    cv2.fillPoly(img, [np.array(pts)], (0,0,0))
    #cv2.imshow('otsu', cv2.resize(img, (int(img.shape[1] * scale),int(img.shape[0] * scale)), interpolation=cv2.INTER_AREA))
    #cv2.waitKey(0)
    cv2.imwrite(img_path,img)
    break
    
#cv2.destroyAllWindows()

In [3]:
img_ori = cv2.imread(file_list[0])
img_result = np.zeros(img_ori.shape[:-1], np.int)
list_gray = []

for img_path in file_list:
    img = cv2.imread(img_path)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)[:, :, 0]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_blur = gray
    gray_blur = cv2.blur(gray, (7, 7), 2)

    #gray_blur = cv2.equalizeHist(gray_blur)
    gray_blur = cv2.bitwise_and(gray_blur, gray_blur, mask=filter_temp)

    gray_value = gray_blur[filter_temp == 255].mean()

    list_gray.append(gray_value)
    ret2, th_otsu = cv2.threshold(gray_blur, 0, 255,
                                  cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    th_otsu = cv2.erode(th_otsu, kernel, iterations=3)
    th_otsu = cv2.dilate(th_otsu, kernel, iterations=3)

    contours, hierarchy = cv2.findContours(th_otsu, cv2.RETR_CCOMP,
                                           cv2.CHAIN_APPROX_NONE)
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        if (cv2.contourArea(c) <= 10000) or (cv2.contourArea(c) >=
                                             100000) or (w > 600) or (h > 300):
            th_otsu = cv2.drawContours(th_otsu, [c],
                                       -1,
                                       0,
                                       thickness=cv2.FILLED)
        #else:
        #    cv2.putText(th_otsu, str(h), (x, y+50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 1, cv2.LINE_AA)

    #cv2.imshow('otsu', cv2.resize(th_otsu, (int(img.shape[1] * scale),int(img.shape[0] * scale)), interpolation=cv2.INTER_AREA))
    #cv2.imshow('gray',  cv2.resize(img, (int(img.shape[1] * scale),int(img.shape[0] * scale)), interpolation=cv2.INTER_AREA))
    #cv2.imshow('hsv',  cv2.resize(hsv, (int(img.shape[1] * scale),int(img.shape[0] * scale)), interpolation=cv2.INTER_AREA))
    #cv2.waitKey(0)
    img_result[th_otsu != 0] += 1

cv2.destroyAllWindows()

In [4]:
open_th = len([i for i in list_gray if i >50 and i < 200]) * 0.75

img_ori = cv2.imread(file_list[0])

img_sheet = np.zeros(img_ori.shape[:-1], np.uint8)
img_sheet[img_result >= open_th] = 255
img_sheet[img_result < open_th] = 0



cv2.imshow('result', cv2.resize(img_sheet, (int(img_ori.shape[1] * scale),int(img_ori.shape[0] * scale)), interpolation=cv2.INTER_AREA))
cv2.waitKey(0)
cv2.destroyAllWindows()


In [5]:
import cv2
import os
import numpy as np

# In[3]:


def nothing(x):
    pass

path = r"D:\MAMAA2\Aging\Code\ImageCpy\check"
file_list = [path + "\\" + i for i in os.listdir(path)]

In [6]:

i = 0


scale = 0.5

targetImgPath = file_list[i]
img = cv2.imread(targetImgPath)
img_hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

winName = 'canny tool'
cv2.namedWindow(winName)

cv2.createTrackbar('LowerH',winName,0,255,nothing)

cv2.createTrackbar('UpperH',winName,255,255,nothing)

cv2.createTrackbar('LowerS',winName,0,255,nothing)

cv2.createTrackbar('UpperS',winName,40,255,nothing)

cv2.createTrackbar('LowerV',winName,160,255,nothing)

cv2.createTrackbar('UpperV',winName,255,255,nothing)


while(1):


    LowerH = cv2.getTrackbarPos('LowerH',winName)

    UpperH = cv2.getTrackbarPos('UpperH',winName)
    
    LowerS = cv2.getTrackbarPos('LowerS',winName)

    UpperS = cv2.getTrackbarPos('UpperS',winName)
    
    LowerV = cv2.getTrackbarPos('LowerV',winName)

    UpperV = cv2.getTrackbarPos('UpperV',winName)
    
    
    lower = np.array([LowerH, LowerS, LowerV])
    upper = np.array([UpperH, UpperS, UpperV])

    
    mask = cv2.inRange(img_hsv, lower, upper)
    result = cv2.bitwise_and(img, img, mask=mask)
    result_hsv = cv2.bitwise_and(img_hsv, img_hsv, mask=mask)
    result_hsv = cv2.bitwise_and(result_hsv, result_hsv, mask=img_sheet)
    
    
    cv2.imshow('ori', cv2.resize(mask, (int(img.shape[1] * scale),int(img.shape[0] * scale)), interpolation=cv2.INTER_AREA))
    cv2.imshow('result', cv2.resize(result, (int(result.shape[1] * scale),int(result.shape[0] * scale)), interpolation=cv2.INTER_AREA))
    cv2.imshow('result_hsv', cv2.resize(result_hsv, (int(result.shape[1] * scale),int(result.shape[0] * scale)), interpolation=cv2.INTER_AREA))

    if cv2.waitKey(1)==ord('c'):
        i+=1
        targetImgPath = file_list[i]
        img = cv2.imread(targetImgPath)
        img_hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
        
    elif cv2.waitKey(1)==ord('z'):
        i-=1
        targetImgPath = file_list[i]
        img = cv2.imread(targetImgPath)
        img_hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
        
    elif cv2.waitKey(1)==ord('q'):
        break



cv2.destroyAllWindows()